In [1]:
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy import units as u

from IPython.display import display

import csv

from targets import TARGETS, ALL_TELESCOPE_FILTERS

In [2]:
MAX_FLAG_THRESHOLD = 0.25
MIN_EXPOSURES = 5
DEFAULT_EXPOSURES = list(range(1, 5)) + list(range(5, 60, 5)) + list(range(60, 600, 60))

In [3]:
PRIORITY = 1000
BINNING = 1
REPEATS = 1
MAX_USES = 3
ALT = 20

In [4]:
for telescope, telescope_filters in ALL_TELESCOPE_FILTERS.items():
    with open(f'data/schedule_{telescope}.csv', 'w') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow((
            'Name', 'RA', 'Dec', 'is timed', 'start time', 'end time',
            'Priority', 'Atoms', 'Binning', 'repeats', 'Max uses', 'Alt limit'
        ))
        for target, filters in TARGETS.items():
            filters = filters & ALL_TELESCOPE_FILTERS[telescope]
            _filters = {}
            for f in filters:
                _filters[f] = []
                for e in DEFAULT_EXPOSURES:
                     _filters[f].append(float(e))
            filters = _filters

            try:
                photometry = Table.read(f'data/{target}.ecsv')
                photometry = photometry[photometry['telescope'] == telescope]
                photometry['flagged'] = (photometry['FLAGS'] & 4) != 0 # In this case we only care about saturation
                for band, group in photometry.to_pandas().groupby('band'):
                    pt = group[['exposure', 'flagged']].pivot_table(
                        columns='flagged',
                        index=['exposure'],
                        aggfunc=lambda x: len(x),
                        fill_value=0,
                    )
                    if True not in pt:
                        pt[True] = 0
                    if False not in pt:
                        pt[False] = 0
                    pt['total'] = pt[True] + pt[False]
                    pt['frac'] = pt[True] / pt['total']

                    flagged_exposures = pt[(pt['total'] >= MIN_EXPOSURES) & (pt['frac'] >= MAX_FLAG_THRESHOLD)].index
                    if len(flagged_exposures) > 0:
                        min_flagged_exposure = flagged_exposures.min()
                        filters[band] = [e for e in filters[band] if e < min_flagged_exposure]
                        
                    successful_exposures = pt[(pt['total'] >= MIN_EXPOSURES) & (pt['frac'] < MAX_FLAG_THRESHOLD)].index
                    if len(successful_exposures) > 0:
                        max_successful_exposure = successful_exposures.max()
                        filters[band] = [e for e in filters[band] if e >= max_successful_exposure]
            except FileNotFoundError:
                pass

            coords = SkyCoord.from_name(target, parse=True)
            atoms = []
            for f, es in filters.items():
                for e in {es[0], es[int(len(es)/2)], es[-1]}: 
                    # Take the unique lowest, middle, and highest exposure times
                    atoms.append(f'{f}:{e}')
            atoms = ';'.join(atoms)
            csv_writer.writerow([
                target, coords.ra.to_string(alwayssign=True, sep=':', unit=u.hour),
                coords.dec.to_string(alwayssign=True, sep=':'), 'FALSE', '',
                '', PRIORITY, atoms, BINNING, REPEATS, MAX_USES, ALT,
            ])